In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, when
import pyspark.sql.functions as f

In [14]:
spark = SparkSession.builder.appName('Lab2').getOrCreate()

In [15]:
nobel = spark.read.csv('nobel.csv', header=True, inferSchema=True)

In [16]:
nobel.printSchema()
print(nobel.count(), len(nobel.columns))

root
 |-- firstname: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- born_country_code: string (nullable = true)
 |-- died_country_code: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- share: integer (nullable = true)
 |-- name_of_university: string (nullable = true)
 |-- city_of_university: string (nullable = true)
 |-- country_of_university: string (nullable = true)
 |-- born_month: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- age_get_prize: integer (nullable = true)

923 14


## Cleaning the data

In [17]:
#dropping unwanted columns
cols = ('born_country_code', 'died_country_code', 'city_of_university', 'born_month', 'age')
nobel = nobel.drop(*cols)

In [18]:
#renamning columns
newColNames = ['fName', 'sName', 'Gender', 'Year', 'Category', 'Share', 'University', 'Country_of_Uni', 'Age_get_Prize']
nobel = nobel.toDF(*newColNames) #toDf only works for flat dataframes

In [19]:
#concatenating firstname and surname separated by space
nobel = nobel.select(concat_ws(' ','fName', 'sName').alias('Name'), *newColNames[2:])
nobel.printSchema()

root
 |-- Name: string (nullable = false)
 |-- Gender: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- Share: integer (nullable = true)
 |-- University: string (nullable = true)
 |-- Country_of_Uni: string (nullable = true)
 |-- Age_get_Prize: integer (nullable = true)



In [20]:
#dropping rows with null values in any columns
nobel = nobel.na.drop('any')
print(nobel.count(), len(nobel.columns))

693 8


## Questions

In [21]:
nobel.filter((nobel.Age_get_Prize > 30) & (nobel.Age_get_Prize < 50)).show(3)

+------------------+------+----+--------+-----+--------------------+---------------+-------------+
|              Name|Gender|Year|Category|Share|          University| Country_of_Uni|Age_get_Prize|
+------------------+------+----+--------+-----+--------------------+---------------+-------------+
|Hendrik A. Lorentz|  male|1902| physics|    2|   Leiden University|the Netherlands|           49|
|     Pieter Zeeman|  male|1902| physics|    2|Amsterdam University|the Netherlands|           37|
|      Pierre Curie|  male|1903| physics|    4|École municipale ...|         France|           44|
+------------------+------+----+--------+-----+--------------------+---------------+-------------+
only showing top 3 rows



In [22]:
nobel = nobel.withColumn('Country_of_Uni', when(nobel.Country_of_Uni == 'the Netherlands', 'Netherlands')\
                         .otherwise(nobel.Country_of_Uni))

In [23]:
nobel.count()

693

In [24]:
nobel.show(12)

+--------------------+------+----+---------+-----+--------------------+--------------+-------------+
|                Name|Gender|Year| Category|Share|          University|Country_of_Uni|Age_get_Prize|
+--------------------+------+----+---------+-----+--------------------+--------------+-------------+
|Wilhelm Conrad Rö...|  male|1901|  physics|    1|   Munich University|       Germany|           56|
|  Hendrik A. Lorentz|  male|1902|  physics|    2|   Leiden University|   Netherlands|           49|
|       Pieter Zeeman|  male|1902|  physics|    2|Amsterdam University|   Netherlands|           37|
|     Henri Becquerel|  male|1903|  physics|    2| École Polytechnique|        France|           51|
|        Pierre Curie|  male|1903|  physics|    4|École municipale ...|        France|           44|
|         Marie Curie|female|1911|chemistry|    1| Sorbonne University|        France|           44|
|       Lord Rayleigh|  male|1904|  physics|    1|Royal Institution...|United Kingdom|     

In [36]:
nobel.agg({'Age_get_Prize': 'avg',
           'Share': 'max'}).show()

+------------------+----------+
|avg(Age_get_Prize)|max(Share)|
+------------------+----------+
|58.683982683982684|         4|
+------------------+----------+



In [37]:
nobel.write.csv('final_nobel.csv')

In [38]:
text = spark.read.text('text.txt')

In [41]:
words = text.withColumn('word', f.explode(f.split(f.col('value'), ' ')))\
        .groupBy('words').count()\
        .sort('count', ascending=False).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `words` cannot be resolved. Did you mean one of the following? [`word`, `value`].;
'Aggregate ['words], ['words, count(1) AS count#748L]
+- Project [value#738, word#741]
   +- Generate explode(split(value#738,  , -1)), false, [word#741]
      +- Relation [value#738] text
